# In this notebook, we use Deep Graph Library (DGL) based classification to identify Fraudulent Medicare providers using data from CMS that has been preprocessed using Data Wrangler

## Train Graph Neural Network using DGL

Graph Neural Networks work by learning representation for nodes or edges of a graph that are well suited for some downstream task. We can model the fraud detection problem as a node classification task, and the goal of the graph neural network would be to learn how to use information from the topology of the sub-graph for each transaction node to transform the node's features to a representation space where the node can be easily classified as fraud or not.

Specifically, we will be using a relational graph convolutional neural network model (R-GCN) on a heterogeneous graph since we have nodes and edges of different types.

### Hyperparameters

To train the graph neural network, we need to define a few hyperparameters that determine properties such as the class of graph neural network models we will be using, the network architecture and the optimizer and optimization parameters. 

Here we're setting only a few of the hyperparameters, to see all the hyperparameters and their default values, see `dgl-fraud-detection/estimator_fns.py`. The parameters set below are:

* **`nodes`** is the name of the file that contains the `node_id`s of the target nodes and the node features.
* **`edges`** is a regular expression that when expanded lists all the filenames for the edgelists
* **`labels`** is the name of the file tha contains the target `node_id`s and their labels
* **`model`** specify which graph neural network to use, this should be set to `r-gcn`

The following hyperparameters can be tuned and adjusted to improve model performance
* **batch-size** is the number nodes that are used to compute a single forward pass of the GNN

* **embedding-size** is the size of the embedding dimension for non target nodes
* **n-neighbors** is the number of neighbours to sample for each target node during graph sampling for mini-batch training
* **n-layers** is the number of GNN layers in the model
* **n-epochs** is the number of training epochs for the model training job
* **optimizer** is the optimization algorithm used for gradient based parameter updates
* **lr** is the learning rate for parameter updates


In [ ]:
!pip install imblearn
!pip install igraph

In [ ]:
import numpy as np 
import pandas as pd
import boto3
import os
import sagemaker
import seaborn as sns
import matplotlib.pyplot as plt
import io
import sklearn
from math import sqrt
from sagemaker import get_execution_role
from sagemaker import RandomCutForest
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import CSVSerializer
from sagemaker.amazon.amazon_estimator import get_image_uri
from sklearn.datasets import dump_svmlight_file  
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, cohen_kappa_score
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.datasets import dump_svmlight_file   
from collections import Counter
import networkx as nx
from igraph import *
import json
from sagemaker_graph_fraud_detection import config
import sys
from os import path
from sagemaker.s3 import S3Downloader 

In [ ]:
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)

In [ ]:
session = sagemaker.Session()
bucket = session.default_bucket()
prefix = 'fraud-detect-demo/graph'
role = get_execution_role()
s3_client = boto3.client("s3")
sys.path
sys.path.append('./sagemaker_graph_fraud_detection/')

**Data Preparation**

Let's start by reading in the entire preprocessed medicare data set prepared for classification

In [ ]:
data=pd.read_csv('features-with-headers-rus.csv')

In [ ]:
data.head()

In [ ]:
data['Fraud'].value_counts()

For DGL, we need to identify the nodes that are used for training, validation and testing (these are called masks)

In [ ]:
train_data_ratio = 0.7
valid_data_ratio = 0.2
n_train = int(data.shape[0]*train_data_ratio)
n_valid = int(data.shape[0]*(train_data_ratio+valid_data_ratio))
valid_ids = data.NPI.values[n_train:n_valid]
test_ids = data.NPI.values[n_valid:]
test_df = data[n_valid:][['NPI','Fraud']]

Save test and validation masks as files - the remaining nodes will be used as the training mask

In [ ]:
with open('validation.csv', 'w') as f:
        f.writelines(map(lambda x: str(x) + "\n", valid_ids))

In [ ]:
with open('test.csv', 'w') as f:
        f.writelines(map(lambda x: str(x) + "\n", test_ids))

Upload all files needed for training to S3 - these include the edges (relation*.csv), the nodes along with the label (tags.csv), the rest of features of each node (features.csv), and the test and validation ids

In [ ]:
s3_client.upload_file('relation_NPI_drug.csv',bucket,'fraud-detect-demo/graph/relation_NPI_drug.csv')

In [ ]:
s3_client.upload_file('relation_NPI_HCPCS.csv',bucket,'fraud-detect-demo/graph/relation_NPI_HCPCS.csv')

In [ ]:
s3_client.upload_file('relation_NPI.csv',bucket,'fraud-detect-demo/graph/relation_NPI.csv')

In [ ]:
s3_client.upload_file('tags.csv',bucket,'fraud-detect-demo/graph/tags.csv')

In [ ]:
s3_client.upload_file('test.csv',bucket,'fraud-detect-demo/graph/test.csv')

In [ ]:
s3_client.upload_file('validation.csv',bucket,'fraud-detect-demo/graph/validation.csv')

In [ ]:
s3_client.upload_file('features.csv',bucket,'fraud-detect-demo/graph/features.csv')

**Now we can begin the prediction of fraud for the test ids using DGL**

Specify the location of the uploaded training data in S3 and the folder to store the model and the output of the predictions

In [2]:
train_data = 'replace with your input S3 uri'
train_output = 'replace with your output S3 uri'

specify the various files that need to be processed

In [ ]:
processed_files = S3Downloader.list(train_data)
print("===== Processed Files =====")
print('\n'.join(processed_files))

Setup the edges to create the graph from the provider relation files and the parameters to train the model 

In [ ]:
edges = ",".join(map(lambda x: x.split("/")[-1], [file for file in processed_files if "relation" in file]))
params = {'nodes' : 'features.csv',
          'edges': 'relation*',
          'labels': 'tags.csv',
          'model': 'rgcn',
          'num-gpus': 1,
          'batch-size': 1024,
          'embedding-size': 1024,
          'n-neighbors': 100,
          'n-layers': 2,
          'n-epochs': 30,
          'optimizer': 'adam',
          'lr': 1e-2
        }

print("Graph will be constructed using the following edgelists:\n{}" .format('\n'.join(edges.split(","))))

### Create and Fit SageMaker Estimator

With the hyperparameters defined, we can kick off the training job. We will be using the Deep Graph Library (DGL), with MXNet as the backend deep learning framework, to define and train the graph neural network. Amazon SageMaker makes it do this with the Framework estimators which have the deep learning frameworks already setup. Here, we create a SageMaker MXNet estimator and pass in our model training script, hyperparameters, as well as the number and type of training instances we want.

We can then `fit` the estimator on the the training data location in S3.

### Hyperparameters

To train the graph neural network, we need to define a few hyperparameters that determine properties such as the class of graph neural network models we will be using, the network architecture and the optimizer and optimization parameters. 

Here we're setting only a few of the hyperparameters, to see all the hyperparameters and their default values, see `dgl-fraud-detection/estimator_fns.py`. The parameters set below are:

* **`nodes`** is the name of the file that contains the `node_id`s of the target nodes and the node features.
* **`edges`** is a regular expression that when expanded lists all the filenames for the edgelists
* **`labels`** is the name of the file tha contains the target `node_id`s and their labels
* **`model`** specify which graph neural network to use, this should be set to `r-gcn`

The following hyperparameters can be tuned and adjusted to improve model performance
* **batch-size** is the number nodes that are used to compute a single forward pass of the GNN

* **embedding-size** is the size of the embedding dimension for non target nodes
* **n-neighbors** is the number of neighbours to sample for each target node during graph sampling for mini-batch training
* **n-layers** is the number of GNN layers in the model
* **n-epochs** is the number of training epochs for the model training job
* **optimizer** is the optimization algorithm used for gradient based parameter updates
* **lr** is the learning rate for parameter updates


In [ ]:
from sagemaker.mxnet import MXNet
from time import strftime, gmtime

estimator = MXNet(
    entry_point='train_dgl_mxnet_entry_point.py',
    source_dir='sagemaker_graph_fraud_detection/dgl_fraud_detection',
    role=role, 
    instance_count=1, 
    instance_type='ml.g4dn.xlarge',
    framework_version="1.6.0",
    py_version='py3',
    hyperparameters=params,
    output_path=train_output,
    code_location=train_output,
    sagemaker_session=session,
)

training_job_name = "{}-{}".format('dgl-classification', strftime("%Y-%m-%d-%H-%M-%S", gmtime()))
print(
    f"You can go to SageMaker -> Training -> Hyperparameter tuning jobs -> a job name started with {training_job_name} to monitor training job status and details."
)
estimator.fit({'train': train_data}, job_name=training_job_name)

Once the training is completed, the training instances are automatically stopped and SageMaker stores the trained model and evaluation results (on the test data) to a location in S3.

### Read the prediction output for the test data
Current training process is transductive setting where the predicting columns of test dataset (not including the target column) are used to construct the graph and thus the test data are included in the training process. At the end of training, the predictions on the test dataset are generated and saved in the **train_output** in the s3 bucket.

In [ ]:
test_output_path = os.path.join(train_output, estimator.latest_training_job.job_name, "output")
!mkdir -p output_dgl_job
!aws s3 cp --recursive $test_output_path output_dgl_job

In [ ]:
import tarfile
  
# open file
tar = tarfile.open(os.path.join("output_dgl_job", "output.tar.gz"), "r:gz")
tar.extractall("output_dgl_job")
tar.close()

In [ ]:
dgl_output = pd.read_csv(os.path.join("output_dgl_job", "preds.csv"))

In [ ]:
y_test = test_df['Fraud'].values.astype('float32')

In [ ]:
y_preds = dgl_output.pred.values.astype('float32')

In [ ]:
def plot_confusion_matrix(y_true, y_predicted):

    cm  = confusion_matrix(y_true, y_predicted)
    # Get the per-class normalized value for each cell
    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    # We color each cell according to its normalized value, annotate with exact counts.
    ax = sns.heatmap(cm_norm, annot=cm, fmt="d")
    ax.set(xticklabels=["non-fraud", "fraud"], yticklabels=["non-fraud", "fraud"])
    ax.set_ylim([0,2])
    plt.title('Confusion Matrix')
    plt.ylabel('Real Classes')
    plt.xlabel('Predicted Classes')
    plt.show()

In [ ]:
print("Balanced accuracy = {:.3f}".format(balanced_accuracy_score(y_test, y_preds)))

In [ ]:
plot_confusion_matrix(y_test, y_preds)

In [ ]:
print(classification_report(
    y_test, y_preds, target_names=['non-fraud', 'fraud']))

### Create and Fit SageMaker Estimator with HPO
In this section we fit the SageMaker Estimator using DGL with HPO.

In [ ]:
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

# Static hyperparameters we do not tune
hyperparameters = {
    'nodes' : 'features.csv',
    'edges': 'relation*',
    'labels': 'tags.csv',
    'model': 'rgcn',
    'num-gpus': 1,
    'n-layers': 2,
    'optimizer': 'adam',
}

# Dynamic hyperparameters we want to tune and their searching ranges. For demonstartion purpose, we skip the architecture search by skipping tunning the hyperparameters such as 'skip_rnn_num_layers', 'rnn_num_layers', and etc.
hyperparameter_ranges = {
    'batch-size': CategoricalParameter([512, 1024, 2048, 10000]),
    'embedding-size': CategoricalParameter([16, 32, 64, 128, 256, 512]),
    'n-neighbors': IntegerParameter(800, 1200),
    'n-epochs': IntegerParameter(10, 17),
    'lr': ContinuousParameter(0.002, 0.1),
}

In [ ]:
objective_metric_name = "Validation F1"
metric_definitions = [{"Name": "Validation F1", "Regex": "Validation F1 (\\S+)"}] #Root Relative Squared Error (RSE): 
objective_type = "Maximize"

In [ ]:
from sagemaker.mxnet import MXNet

estimator_tuning = MXNet(
    entry_point='train_dgl_mxnet_entry_point.py',
    source_dir='sagemaker_graph_fraud_detection/dgl_fraud_detection',
    role=role, 
    instance_count=1, 
    instance_type='ml.g4dn.xlarge',
    framework_version="1.6.0",
    py_version='py3',
    hyperparameters=params,
    output_path=train_output,
    code_location=train_output,
    sagemaker_session=session,
)

In [ ]:
import time

tuning_job_name = "{}-{}".format('dgl-classification-tuning', strftime("%Y-%m-%d-%H-%M-%S", gmtime()))
print(
    f"You can go to SageMaker -> Training -> Hyperparameter tuning jobs -> a job name started with {tuning_job_name} to monitor HPO tuning status and details.\n"
    f"Note. You will be unable to successfully run the following cells until the tuning job completes. This step may take around 2 hour."
)

tuner = HyperparameterTuner(
    estimator_tuning,  # using the estimator defined in previous section
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=20,
    max_parallel_jobs=2,
    objective_type=objective_type,
    base_tuning_job_name = tuning_job_name,
)

start_time = time.time()

tuner.fit({'train': train_data})

hpo_training_job_time_duration = time.time() - start_time

In [ ]:
import boto3
sm_client = boto3.Session().client("sagemaker")

tuning_job_name = tuner.latest_tuning_job.name
tuning_job_name

In [ ]:
tuning_job_result = sm_client.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name
)

status = tuning_job_result["HyperParameterTuningJobStatus"]
if status != "Completed":
    print("Reminder: the tuning job has not been completed.")

job_count = tuning_job_result["TrainingJobStatusCounters"]["Completed"]
print("%d training jobs have completed" % job_count)

is_minimize = (
    tuning_job_result["HyperParameterTuningJobConfig"]["HyperParameterTuningJobObjective"]["Type"]
    != "Minimize"
)
objective_name = tuning_job_result["HyperParameterTuningJobConfig"][
    "HyperParameterTuningJobObjective"
]["MetricName"]

In [ ]:
tuner_analytics = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)

full_df = tuner_analytics.dataframe()

if len(full_df) > 0:
    df = full_df[full_df["FinalObjectiveValue"] > -float("inf")]
    if len(df) > 0:
        df = df.sort_values("FinalObjectiveValue", ascending=False)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest": min(df["FinalObjectiveValue"]), "highest": max(df["FinalObjectiveValue"])})
        pd.set_option("display.max_colwidth", -1)  # Don't truncate TrainingJobName
    else:
        print("No training jobs have reported valid results yet.")

df

### Read the prediction output for the test dataset from the best tuning job

In [ ]:
import os
df = df[df["TrainingJobStatus"] == "Completed"] # filter out the failed jobs
output_path_best_tuning_job = os.path.join(train_output, df["TrainingJobName"].iloc[0], "output")
print(output_path_best_tuning_job)

In [ ]:
!mkdir -p output_dgl_best_tuning_job
!aws s3 cp --recursive $output_path_best_tuning_job output_dgl_best_tuning_job

In [ ]:
import tarfile
  
# open file
tar = tarfile.open(os.path.join("output_dgl_best_tuning_job", "output.tar.gz"), "r:gz")
tar.extractall("output_dgl_best_tuning_job")
tar.close()

In [ ]:
dgl_output = pd.read_csv(os.path.join("output_dgl_best_tuning_job", "preds.csv"))

In [ ]:
y_preds = dgl_output.pred.values.astype('float32')

In [ ]:
plot_confusion_matrix(y_test, y_preds)

In [ ]:
print(classification_report(
    y_test, y_preds, target_names=['non-fraud', 'fraud']))

In [ ]:
print("Balanced accuracy = {:.3f}".format(balanced_accuracy_score(y_test, y_preds)))

## Clean Up


After you are done using this notebook, delete the model artifacts and other resources to avoid any incurring charges.

**Caution**: You need to manually delete resources that you may have created while running the notebook, such as Amazon S3 buckets for model artifacts, training datasets, processing artifacts, and Amazon CloudWatch log groups.
